In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
DROP TABLE IF EXISTS Disease;
DROP TABLE IF EXISTS Symptom;
DROP TABLE IF EXISTS Cause;
DROP TABLE IF EXISTS Similarity;

CREATE TABLE Disease (
    name VARCHAR(512) NOT NULL,
    occurrences INT NOT NULL,
    PRIMARY KEY (name)
) AS SELECT * FROM CSVREAD('../data/ncomms-disease.csv', null, 'charset=UTF-8 fieldSeparator=|');

CREATE TABLE Symptom (
    name VARCHAR(512) NOT NULL,
    occurrences INT NOT NULL,
    PRIMARY KEY (name)
) AS SELECT * FROM CSVREAD('../data/ncomms-symptom.csv', null, 'charset=UTF-8 fieldSeparator=|');

CREATE TABLE Cause (
    symptom VARCHAR(512) NOT NULL,
    disease VARCHAR(512) NOT NULL,
    occurrences INT NOT NULL,
    score DOUBLE NOT NULL,
    PRIMARY KEY (disease, symptom),
    FOREIGN KEY (disease)
        REFERENCES Disease(name)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY (symptom)
        REFERENCES Symptom(name)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
) AS SELECT * FROM CSVREAD('../data/ncomms-cause.csv', null, 'charset=UTF-8 fieldSeparator=|');

CREATE TABLE Similarity (
    disease_from VARCHAR(512) NOT NULL,
    disease_to VARCHAR(512) NOT NULL,
    score DOUBLE NOT NULL,
    PRIMARY KEY (disease_from, disease_to),
    FOREIGN KEY (disease_from)
        REFERENCES Disease(name)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
    FOREIGN KEY (disease_to)
        REFERENCES Disease(name)
            ON DELETE NO ACTION
            ON UPDATE NO ACTION,
) AS SELECT * FROM CSVREAD('../data/ncomms-similarity.csv', null, 'charset=UTF-8 fieldSeparator=|');